In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This program is dedicated to the public domain under the CC0 license.

"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging
import re
from telegram.ext import Updater, ConversationHandler, CommandHandler, MessageHandler, Filters
import random
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define global variables
CUSTOMER_NUM = None
MENU, AUTH, AUTH_BIRTH, AC_INFO,PRODUCT = range(5)


# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


#Pulling data from Google Spreadsheets
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'C:\\Users\\user\Desktop\\credentials.json', scope) # Your json file here

gc = gspread.authorize(credentials)

wks = gc.open("card_data").sheet1

data = wks.get_all_values()
headers = data.pop(0)

card_data = pd.DataFrame(data, columns=headers)
#card_data=pd.read_csv('C:\\Users\\user\Desktop\\card_data.csv')

card_data['card_no']=card_data['card_no'].astype(str)
card_data['card_no']=card_data['card_no'].apply(lambda x: x.zfill(4))
list1=card_data['card_no']

greetings_data=pd.read_csv('C:\\Users\\user\Desktop\\list_greetings.csv')
listing=greetings_data['List_greets']
listing = [x for x in listing if x != 'nan']
listing=[re.sub('[^a-zA-Z0-9]+', '', _) for _ in listing]
list2 = [i for n, i in enumerate(listing) if i not in listing[:n]]
list3=greetings_data['list_ends'].astype(str)
list3 = [x for x in list3 if x != 'nan']

# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update, context):
    
    first_name = update.message.chat.first_name if len(update.message.chat.first_name) > 0 else ""
    """Send a message when the command /start is issued."""
    
    #print(first_name)
    n = random.randint(0,10)

    if n in [0,1,2]:
        update.message.reply_text(f'Hi {first_name}! I am Penny \nI can help you on the following things \nPress 1 for Account Info \nPress 2 for Product Features \nPress 3 for Spend Offers \nPress 4 for Credit Card Application Link \nPress 0 for Exit')
    elif n in [3,4,5]:
        update.message.reply_text(f'Hey {first_name}. I am Penny \nI can help you on the following things \nPress 1 for Account Info \nPress 2 for Product Features \nPress 3 for Spend Offers \nPress 4 for Credit Card Application Link \nPress 0 for Exit')
    elif n in [6,7,8]:
        update.message.reply_text(f'Hola {first_name}, I am Penny \nI can help you on the following things \nPress 1 for Account Info \nPress 2 for Product Features \nPress 3 for Spend Offers \nPress 4 for Credit Card Application Link \nPress 0 for Exit')
    elif n in [9,10]:
        update.message.reply_text(f"Hello {first_name}, My Name is Penny \nI can help you on the following things \nPress 1 for Account Info \nPress 2 for Product Features \nPress 3 for Spend Offers \nPress 4 for Credit Card Application Link \nPress 0 for Exit")
    return MENU
        

def help(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text("Following are the commands I understand - \n'/start' or any valid greeting like '/Hi' to trigger Dollar\n1 to get account info (Login required) \n2 for any product details \n3 to receive our latest and hottest offers \n4 to receive our card application landing page etc \nEnjoy the bot and be easy on me, I am here to just help you!")

def auth(update, context):
    ac_input = update.message.text
    
    global CUSTOMER_NUM
    
    if (len(str(ac_input)) == 4) and str(ac_input).isdigit():
        if ac_input in card_data['card_no'].values:      
            card_data1=card_data[card_data['card_no']==ac_input]
            name=card_data1.iloc[0,2]
            update.message.reply_text("Hello "+ name)
            update.message.reply_text("Please enter your Year of Birth")
            CUSTOMER_NUM = ac_input
            return AUTH_BIRTH
        else:
            update.message.reply_text("This card does not exist in our database, kindly re-enter a valid one")
            return AUTH
    else:
        update.message.reply_text("Oops! Looks like you did not enter 4 digits")
        return AUTH
    
    
def auth_by_birth(update, context):
    global CUSTOMER_NUM
    ac_input = update.message.text
    user_card_data=card_data[card_data['card_no']==CUSTOMER_NUM]
    year=user_card_data.iloc[0,1]
    
    if (len(str(ac_input)) == 4) and str(ac_input).isdigit():
        if str(int(ac_input)) == year:                  
            update.message.reply_text("Logged in Successfully!")
            update.message.reply_text("Welcome to the Authenticated Account Information session.\n Press 1 for Balance \n Press 2 for Reward Balance \n Press 3 for Card Limit \n Press 4 for Last Spend \n Press 5 for Spend Insights \n Press 0 to go back main menu")
            return AC_INFO
        else:
            print(year)
            update.message.reply_text("Incorrect year of birth, please try to input again.")
            return AUTH_BIRTH
    else:
        update.message.reply_text("Oops! Looks like you did not a valid year number, please try to input again.")
        return AUTH_BIRTH

def account_info_page(update, context):
    """
    (1) page
    """
    global CUSTOMER_NUM
    
    user = update.message.from_user
    user_card_data=card_data[card_data['card_no']==CUSTOMER_NUM]

    if update.message.text == '0':
        update.message.reply_text("Back to main menu")
        update.message.reply_text("Main Menu\nPress 1 for Account Info \nPress 2 for Product Features \nPress 3 for Spend Offers \nPress 4 for Credit Card Application Link \nPress 5 for Spend Insights \nPress 0 for Exit")
        return MENU
                                  
    elif update.message.text == '1':
        update.message.reply_text(f"Balance: {user_card_data.iloc[0,3]}. You can input another number to check info or input 0 to exit")
    
    elif update.message.text == '2':
        update.message.reply_text(f"Reward Balance: {user_card_data.iloc[0,4]}. You can input another number to check info or input 0 to exit")
    
    elif update.message.text == '3':
        update.message.reply_text(f"Limit: {user_card_data.iloc[0,5]}. You can input another number to check info or input 0 to exit")
    
    elif update.message.text == '4':
        update.message.reply_text(f"Last Spent: {user_card_data.iloc[0,6]}. You can input another number to check info or input 0 to exit")
    
    elif update.message.text == '5':
        update.message.reply_text(f"Here are your card spend insights - \n\n{user_card_data.iloc[0,7]}.\n\nYou can input another number to check info or input 0 to exit")
        
    return AC_INFO

    
def product_page(update, context):
    """
    (2) page
    """
    selected_product = update.message.text
    if selected_product[1:].lower() == 'premier':
        update.message.reply_text("Following are the premier benefits - \n1. Earn 1 AirMile for every $1 spend \n2.Complimentary access to a select number of airport lounges worldwide \n3. 'Buy 1 Get 1 Free' movie offers \nand many more great offers \n\nEligibility - \nMonthly Salary of $10,000 or \nTotal Relationship Balance of $100,000 \n\n Type '/Mass' to view mass offer too if you find it not sutiable for you.\n Type 0 to go back main menu")
        return PRODUCT
    elif selected_product[1:].lower() == 'mass':
        update.message.reply_text("Following are the mass benefits - \n1. Earn 1 AirMile for every $4 spend \n2.Get up to 12% off across 985,000+ hotels and vacation rentals globally at Agoda \n3. 'Buy 1 Get 1 Free' movie offers \nand many more great offers \n\nEligibility - \nMonthly Salary of $2,000 or \nTotal Relationship Balance of $10,000. \n\n Type '/Premier' to view Premier offer for better benefits!\n Type 0 to go back main menu")
        return PRODUCT
    else:
        update.message.reply_text("Back to main menu")
        update.message.reply_text("Main Menu\nPress 1 for Account Info \nPress 2 for Product Features \nPress 3 for Spend Offers \nPress 4 for Credit Card Application Link \nPress 0 for Exit")
        return MENU

def spending_offer_page(update, context):
    """
    (3) page
    """
    update.message.reply_text("Here are some our great spend offers \n1. Dine Out to get 5% cashback\n2. Spend $100 or higher Online and get flat 1% rebate\n3. Now Pay your kids school fees and repay in easy monthly installments\n4. Transfer balance from other banks to enjoy preferential rates\n5. Enjoy No Cost Payment Plan when you buy your new iphone\nHappy Shopping with our cards!")
    return 
    
def apply_card_page(update, context):
    """
    (4) page
    """
    update.message.reply_text("https://www.hsbc.com.hk/premier/")
    return 

def endnote(update, context):
    update.message.reply_text("It was great to speak to you, \nI hope to see you again soon!")
    return

    
def echo(update, context):
    """Echo the user message.""" 
    #print(type(update.message.text))
    string=update.message.text
    string1=string[1:]
    string2=string[1:5]
    #print(str(string1).isdigit())
    #print(len(str(string1)))
    if str(string1).isdigit() and (len(str(string1)) != 4):
        n = random.randint(0,3)
        if n in [1,3]:
            update.message.reply_text("Looks like you did not enter a valid last 4 digit card number, kindly re-enter a 4 digit number")
        elif n in [0,2]:
            update.message.reply_text("If you are looking for Acount Info for "+ string2 + "\nJust tap on this /"+ string2)
    else:
        n = random.randint(0,3)
        if n==0:
            update.message.reply_text("Apologies I don't understand your query \nType '/help' to understand the bot functionalities")
        elif n==1:
            update.message.reply_text("Sorry I don't understand your input \nType '/help' to understand the bot functionalities")
        elif n==2:
            update.message.reply_text("Could not process your input \nType '/help' to understand my capabilities")
        elif n==3:
            update.message.reply_text("I can't answer this query \nType '/help' to understand my capabilities")
            
def menu(update,context):
    """
    Main menu function.
    """
    user = update.message.from_user
    if update.message.text == '1':
        STATE = AC_INFO
        update.message.reply_text("Great! So you want to know your Account Information, can you help me with the last four digits of your card number")
        return AUTH
    
    elif update.message.text == '2':
        update.message.reply_text("Fantastic! which product are you looking for? \nFor Premium Products Type '/Premier'\nFor Everyday Banking Type '/Mass'")
        return PRODUCT
    
    elif update.message.text == '3':
        spending_offer_page(update, context)
        return MENU
    
    elif update.message.text == '4':
        apply_card_page(update, context)
        return MENU
    
    elif update.message.text == '0':
        endnote(update, context)
        return ConversationHandler.END
    
    else:
        return MENU
            
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    
    global CUSTOMER_NUM
    updater = Updater("1722147162:AAHIg4O7ni32DyutKi1GcXW4M-RNHcUfAbg", use_context=True)


    # Get the dispatcher to register handlers
    dp = updater.dispatcher
    
    product_reges= re.compile(r'^(/mass|/premier|0)$', re.IGNORECASE)
    
    
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler(list2, start)],

        states={
            MENU: [MessageHandler(Filters.regex(r'^\d+$'), menu),CommandHandler(list2, start)],
            AUTH: [MessageHandler(Filters.regex(r'^\d+$'), auth)],
            AUTH_BIRTH: [MessageHandler(Filters.regex(r'^\d+$'), auth_by_birth)],
            AC_INFO: [MessageHandler(Filters.regex(r'^\d+$'), account_info_page)],
            PRODUCT: [MessageHandler(Filters.regex(product_reges), product_page)],
        },
        
        fallbacks=[CommandHandler('help', help)
                  ]
    )
    dp.add_handler(conv_handler)

    # on noncommand i.e message - echo the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, echo))
    
    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    updater.idle()

/
if __name__ == '__main__':
    main()